# **Getting The Data**

In [ ]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"yasnatalebi","key":"b34a861570f8248b7154c1502d057620"}'}

In [ ]:
import os
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle competitions download -c rossmann-store-sales

!unzip -o rossmann-store-sales.zip

  0% 0.00/6.99M [00:00<?, ?B/s]
100% 6.99M/6.99M [00:00<00:00, 480MB/s]
Archive:  rossmann-store-sales.zip
  inflating: sample_submission.csv   
  inflating: store.csv               
  inflating: test.csv                
  inflating: train.csv               


In [ ]:
import pandas as pd
from sqlalchemy import create_engine
from datetime import datetime

# ---------- Load raw CSV files ----------
sales_df = pd.read_csv('train.csv')
store_df = pd.read_csv('store.csv')

# ---------- Merge store info into sales ----------
merged_df = sales_df.merge(store_df, on='Store', how='left')

# ---------- Create dim_store ----------
dim_store = store_df.copy()
dim_store.rename(columns={
    'Store': 'store_id',
    'StoreType': 'store_type',
    'Assortment': 'assortment_type',
    'CompetitionDistance': 'competition_distance',
    'CompetitionOpenSinceMonth': 'competition_open_month',
    'CompetitionOpenSinceYear': 'competition_open_year',
    'Promo2': 'promo2',
    'Promo2SinceWeek': 'promo2_since_week',
    'Promo2SinceYear': 'promo2_since_year',
    'PromoInterval': 'promo_interval'
}, inplace=True)

# ---------- Create dim_date ----------
# Convert to datetime
merged_df['Date'] = pd.to_datetime(merged_df['Date'])

# Build date dimension
date_range = pd.date_range(start=merged_df['Date'].min(), end=merged_df['Date'].max())
dim_date = pd.DataFrame({
    'date_id': date_range,
    'day': date_range.day,
    'month': date_range.month,
    'year': date_range.year,
    'weekday': date_range.weekday,
    'is_weekend': date_range.weekday >= 5
})

# ---------- Create fact_sales ----------
fact_sales = merged_df[['Store', 'Date', 'Sales', 'Customers', 'Promo', 'Open', 'SchoolHoliday', 'StateHoliday']].copy()
fact_sales.rename(columns={
    'Store': 'store_id',
    'Date': 'date_id',
    'Sales': 'sales',
    'Customers': 'customers',
    'Promo': 'promo',
    'Open': 'open',
    'SchoolHoliday': 'school_holiday',
    'StateHoliday': 'state_holiday'
}, inplace=True)

/tmp/ipython-input-579147009.py:6: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  sales_df = pd.read_csv('train.csv')


In [ ]:
# ---------- Data Quality Checks ----------
print("\n🔍 Running Data Quality Checks...")

# Nulls in fact_sales
print("\n🧪 Nulls in fact_sales:")
print(fact_sales.isnull().sum())

# Negative or zero sales
neg_sales = fact_sales[fact_sales['sales'] <= 0].shape[0]
print(f"\n📉 Records with sales <= 0: {neg_sales}")

# Negative customers
neg_customers = fact_sales[fact_sales['customers'] < 0].shape[0]
print(f"\n👥 Records with negative customers: {neg_customers}")

# Invalid dates
invalid_dates = fact_sales[
    (fact_sales['date_id'] > '2025-01-01') |
    (fact_sales['date_id'] < '2010-01-01')
].shape[0]
print(f"\n📅 Records with out-of-range dates: {invalid_dates}")


🔍 Running Data Quality Checks...

🧪 Nulls in fact_sales:
store_id          0
date_id           0
sales             0
customers         0
promo             0
open              0
school_holiday    0
state_holiday     0
dtype: int64

📉 Records with sales <= 0: 172871

👥 Records with negative customers: 0

📅 Records with out-of-range dates: 0


In [ ]:
# Nulls in dim_store
print("\n🏬 Nulls in dim_store:")
print(dim_store.isnull().sum())

# Negative competition distance
neg_dist = dim_store[dim_store['competition_distance'] < 0].shape[0]
print(f"\n📏 Records with negative competition distance: {neg_dist}")


🏬 Nulls in dim_store:
store_id                    0
store_type                  0
assortment_type             0
competition_distance        3
competition_open_month    354
competition_open_year     354
promo2                      0
promo2_since_week         544
promo2_since_year         544
promo_interval            544
dtype: int64

📏 Records with negative competition distance: 0


In [ ]:
import numpy as np

# Handling Missing Values
dim_store.loc[
    dim_store['competition_distance'].isnull(),
    ['competition_distance', 'competition_open_month', 'competition_open_year']
] = 0

median_month = dim_store['competition_open_month'].median()
median_year = dim_store['competition_open_year'].median()

dim_store.loc[
    (dim_store['competition_distance'] != 0) &
    (dim_store['competition_open_month'].isnull()),
    'competition_open_month'
] = median_month

dim_store.loc[
    (dim_store['competition_distance'] != 0) &
    (dim_store['competition_open_year'].isnull()),
    'competition_open_year'
] = median_year


In [ ]:
dim_store.fillna(0, inplace=True)
dim_store.isnull().sum()

,0
store_id,0
store_type,0
assortment_type,0
competition_distance,0
competition_open_month,0
competition_open_year,0
promo2,0
promo2_since_week,0
promo2_since_year,0
promo_interval,0


In [ ]:
# Duplicates in dim_date
dup_dates = dim_date.duplicated(subset=['date_id']).sum()
print(f"\n🔁 Duplicate records in dim_date: {dup_dates}")

print("\n✅ Data quality checks complete.")


🔁 Duplicate records in dim_date: 0

✅ Data quality checks complete.


In [ ]:
# ---------- Connect to DB ----------
engine = create_engine('sqlite:///rossmann_dw.db')

# ---------- Load dimension tables ----------
dim_store.to_sql('dim_store', engine, if_exists='replace', index=False)
dim_date.to_sql('dim_date', engine, if_exists='replace', index=False)

# ---------- Load fact table ----------
fact_sales.to_sql('fact_sales', engine, if_exists='replace', index=False)

print("✅ ETL completed. All tables loaded to data warehouse.")

✅ ETL completed. All tables loaded to data warehouse.


In [ ]:
!cp /content/rossmann_dw.db /content/drive/MyDrive/